### 掛接 google drive雲端硬碟
### 引入函式庫

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 安裝必要套件
! sudo apt-get -y update
! sudo apt-get -y install libboost-all-dev libhdf5-serial-dev libzmq3-dev libopencv-dev python-opencv git vim
! sudo pip install Cython scikit-image keras==2.1.1 configobj IPython
! sudo apt-get -y install python-tk python3-pip python3-dev python3-tk cuda-toolkit-9-0
! sudo pip3 install tensorflow-gpu==1.12.0
! sudo pip3 install keras==2.1.1 Cython scikit-image pandas zmq h5py opencv-python IPython configobj cupy-cuda90
!pip install flask-ngrok
!pip install line-bot-sdk

# 引入函式庫
import os
import sys
import cv2
import glob
import json
import time
import datetime
import PIL
import PIL.Image
import numpy as np
import pandas as pd
from google.colab import files
from google.colab.patches import cv2_imshow
from sklearn.externals import joblib
from scipy.special import boxcox1p, inv_boxcox1p

import random
import configparser
from pathlib import Path
from __future__ import unicode_literals
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import LineBotApiError, InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, PostbackEvent, TemplateSendMessage, ButtonsTemplate, PostbackTemplateAction, MessageTemplateAction

# 自訂圖片縮放大小功能
def img_resize(image):
    height, width = image.shape[0], image.shape[1]
    # 設置新的圖片分辨率框架
    width_limit = 2000
    height_limit = 1440
    # 判斷圖片的長寬比率
    if width / height >= width_limit / height_limit:
        img_new = cv2.resize(image, (width_limit, int(height * width_limit / width)))
    else:
        img_new = cv2.resize(image, (int(width * height_limit / height), height_limit))
    return img_new

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,415 kB]
Hit:12 http://ppa.launchpad.net

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Line Bot 上傳圖片、輸入基本資料
### 計算三圍、體脂率

In [4]:
app = Flask(__name__)
# LINE 聊天機器人的基本資料
config = configparser.ConfigParser()
config.read('/content/drive/Shareddrives/AI_Python/ishape/ai/demo/config.ini')
line_bot_api = LineBotApi('d/oSZG6JylDacqi8DsLI9SjlINe9PDbgHTwhsKxm0hxv4wc5PnXc34PaE+Uebb0VzWXJoQLuLwLwTlsW4JdhckKK6aIk2mjEh8y3xRSBtmBIo0eDbHjrSThw8lHSkdDvmOtYszTk2zI46bQkCDSmWgdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('fb3cf7aa9ca017e7fd96683be7de56da')
run_with_ngrok(app)   #starts ngrok when the app is run

message_ls = []
body_datas = {'gender':'', 'age':0, 'height':0.0, 'weight':0.0, 'bmi':0.0, 'img':''}

# 接收 LINE 的資訊
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    try:
        # print(body, signature)
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 接收 LINE圖片，並要求點選性別
@handler.add(MessageEvent, message=ImageMessage)
def handle_image(event):
    message_id = event.message.id
    user_id = event.source.user_id

    # 從 message_id 取得圖片的binary data
    message_content = line_bot_api.get_message_content(message_id)

    with open(Path(f"/content/{message_id}.jpg").absolute(), "wb") as f:
        # 1024 byte 逐一寫入
        for chunk in message_content.iter_content():
            f.write(chunk)
    
    filename = glob.glob("/content/" + message_id + ".jpg")[0]
    if filename:
        try:
            line_bot_api.push_message(user_id, TextSendMessage(text= '圖片上傳成功'))
        except LineBotApiError as e:
            print('error:', e)
            raise e
        img = cv2.imread(filename)
        img = img_resize(img)
        cv2.imwrite(filename, img)
        body_datas['img'] = img
        body_datas['img_name'] = message_id

        # 上傳圖片後需點選性別
        line_bot_api.reply_message(
            event.reply_token,
            TemplateSendMessage(
                alt_text='Buttons template, only avalible on mobile phone',
                template=ButtonsTemplate(
                    title='請選擇性別',
                    text='Gender',
                    actions=[
                        PostbackTemplateAction(label='♂男性', data = 'gender&m'),
                        PostbackTemplateAction(label='♀女性', data = 'gender&f')
                    ]
                )
            )
        )

# 接收性別資料，並回傳年齡資料請求
@handler.add(PostbackEvent)
def postback_text(event):
    gender = str(str(event.postback.data)[-1])
    body_datas['gender'] = gender
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請輸入年齡"))


# 接收 LINE 輸入的身體資料
@handler.add(MessageEvent, message=TextMessage)
def handle_text(event):
    message_id = event.message.id
    content = event.message.text
    message_ls.append(content)
    if len(message_ls) == 1:
        age = float(event.message.text)
        body_datas['age'] = age
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="請輸入身高(cm)")
        )
    elif len(message_ls) == 2:
        height = float(event.message.text)
        body_datas['height'] = height
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="請輸入體重(kg)")
        )
    elif bool(body_datas['age']) and bool(body_datas['height']) and len(message_ls) > 2:
        weight = float(event.message.text)
        body_datas['weight'] = weight
        line_bot_api.push_message(event.source.user_id, TextSendMessage(text= '體脂率分析中...'))
        bfp_analysis(event.source.user_id, body_datas)

# 使用者輸入必要資料後，開始體脂率分析，並回傳分析結果
def bfp_analysis(user_id, body_datas):
    t_start = time.time()
    img = body_datas['img']
    img_name = body_datas['img_name']
    gender = body_datas['gender']
    age = body_datas['age']
    height = body_datas['height']
    weight = body_datas['weight']
    bmi =  round(weight / ((height / 100) ** 2), 3)
    body_datas['bmi'] = bmi

    # 移動目前讀取的資料夾目錄到 human-part-segmentation 資料夾
    % cd "/content/drive/Shareddrives/AI_Python/ishape/ai/demo/CDCL-human-part-segmentation"
    # 建立已處理資料夾 processed
    dirname = "/content/processed/"
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    # 從上傳的正面全身照圖片，劃分人體主要部分，產生人體部位圖
    ! python inference_7parts.py --input_folder='/content/' --output_folder='/content/processed/'
    segment_img = glob.glob('/content/processed/seg_' + img_name + '.jpg' )
    # PIL.Image.open(str(segment_img[0]))

    # 從上傳的正面全身照圖片，取得人體骨架，產生骨架圖
    !python inference_15parts_skeletons_json.py --input_folder='/content/' --output_folder='/content/processed'
    sk_img = glob.glob('/content/processed/sk_' + img_name + '.jpg' )
    # PIL.Image.open(str(sk_img[0]))

    sys.path.append("/content/drive/Shareddrives/AI_Python/ishape/ai/demo/")
    from body_width import BodyWidth
    # 原始圖片、人體區塊圖、人體骨架座標點
    org_img = img
    seg_img = cv2.imread(str(segment_img[0]))
    img_json = glob.glob('/content/processed/' + img_name + '.json')[0]
    jf = {}
    with open(img_json, "r", encoding="utf-8") as f:
        jf = json.load(f)

    bw = BodyWidth(org_img, seg_img, jf, height, gender)

    # 依比例計算寬度
    body_width_df = {
        "org_img": [],
        "gender":[],
        "age":[],
        "height": [],
        "weight": [],
        "width_per_px": [],
        "waist_width_px": [],
        "hip_width_px": [],
        "chest_width_px":[],
        "waist_width_cm": [],
        "hip_width_cm": [],
        "chest_width_cm":[]
    }
    ankle_y = bw.get_ankle_y()
    shoulder_y = bw.get_shoulder_y()
    waist_y = bw.get_waist_y(shoulder_y, ankle_y)
    hip_y = bw.get_hip_y()

    # 腰寬、臀寬
    wpos1, wpos2, ww_px = bw.cal_width(waist_y)
    hpos1, hpos2, hw_px = bw.cal_width(hip_y)

    # 胸寬
    chw_list = []
    r = int((waist_y - shoulder_y) * 0.15)
    for i in range(shoulder_y + r, waist_y - 2*r):
        p1, p2, w = bw.cal_width(i)
        chw_list.append((w, p1, p2))
    chw = max(chw_list)
    chw_px, chpos1, chpos2 = chw

    cm_per_px = bw.cal_cm_per_px(shoulder_y, ankle_y, bw.sh_prop, bw.ak_prop)
    chw_cm = round(chw_px * cm_per_px, 3)
    ww_cm = round(ww_px * cm_per_px, 3)
    hw_cm = round(hw_px * cm_per_px, 3)
    # print("胸寬:", chw_cm, "公分")
    # print("腰寬:", ww_cm, "公分")
    # print("臀寬:", hw_cm, "公分")

    user_info = {
    'age': age,
    'height': height,
    'weight': weight,
    'chest_width_cm': chw_cm,
    'waist_width_cm': ww_cm,
    'hip_width_cm': hw_cm,
    'bmi': bmi
    }
    user_df = pd.DataFrame.from_dict(user_info,orient='index').T
    predict_df = pd.DataFrame.from_dict(user_info,orient='index').T
    pd.set_option("display.max_columns", None)
    # random forest 計算胸圍
    model_chest = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/chest/2021-06-03_05:18/RandomForestRegressor/pre_chest.h5')
    user_df['chest'] = model_chest.predict(predict_df)
    # random forest 計算腰圍
    predict_df = pd.DataFrame.from_dict(user_info,orient='index').T
    model_waist = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/waist/2021-06-03_09:03/RandomForestRegressor/pre_waist.h5')
    user_df['waist'] = model_waist.predict(predict_df)
    # random forest 計算臀圍
    predict_df = pd.DataFrame.from_dict(user_info,orient='index').T
    model_hips = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/hips/2021-06-03_09:30/RandomForestRegressor/pre_hips.h5')
    user_df['hips'] = model_hips.predict(predict_df)
    # Partial least squares regression 計算體脂率
    predict_df = user_df[['age', 'height', 'weight', 'chest', 'waist', 'hips', 'bmi']]
    model_bfp = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step3/PLSRegression.h5')
    user_df['pre_bfp'] = model_bfp.predict(predict_df).round(2)
    bfp = user_df['pre_bfp'].values[0]

    bfp_report = ""
    standard_weight =  round(( user_df['height'] - 80 ) * 0.7, 2) # 標準體重
    wh_ratio = round(user_df['waist'] / user_df['hips'],2)*100 # 腰臀比
    chest = round(user_df['chest'].values[0], 2)
    waist = round(user_df['waist'].values[0], 2)
    hips = round(user_df['hips'].values[0], 2)
    now = datetime.datetime.now().strftime('%Y/%m/%d') # 當前時間(注意為伺服器時間)

    bfp_report += str(now) + '\n' + 'iShape分析報告\n'
    bfp_report += '理想體重 : ' + str(standard_weight.values[0]) + ' 公斤\n'
    bfp_report += '-------------------------\n'
    bfp_report += '胸圍 : ' + str(chest) + ' 公分\n'
    if waist > 90 :
        bfp_report += '腰圍 : ' + str(waist) + ' 公分\n'
        bfp_report += '已超過衛福部規定\n男性健康腰圍\n'
        bfp_report += '請注意內臟脂肪過高\n有心血管疾病、糖尿病、\n高血壓等風險\n'
    else:
        bfp_report += '腰圍 : ' + str(waist) + ' 公分 \n屬於健康範圍\n請繼續保持\n'

    bfp_report += '臀圍 : ' + str(hips) + ' 公分\n'
    bfp_report += '腰臀比 : ' + str(wh_ratio.values[0]) + '%\n'
    bfp_report += '-------------------------\n'
    bfp_report += '體脂率 : ' + str(bfp) + ' %\n'
    if bfp > 25:
        bfp_report += '您的體脂率已超過25%\n屬於過胖，請多運動\n並減少精緻食物攝取\n'
    elif 15 < bfp <= 25:
        bfp_report += '您的體脂率為健康範圍\n請繼續保持\n'
    elif bfp < 5:
        bfp_report += '您的體脂率低於5%\n可能影響身體機能\n請多多攝取蛋白質與堅果類\n'
    bfp_report += '-------------------------'
    line_bot_api.push_message(user_id, TextSendMessage(text=bfp_report))
  
    t_end = time.time()
    print("計算時間:", round(t_end - t_start, 2), "秒")
    body_datas.clear()
    message_ls.clear()

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8a50b69d2445.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Jun/2021 11:47:14] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 11:47:50] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 11:47:53] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 11:47:59] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 11:48:10] "POST /callback HTTP/1.1" 200 -


/content/drive/Shareddrives/AI_Python/ishape/ai/demo/CDCL-human-part-segmentation
2021-06-12 11:48:16.638285: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
start processing...
2021-06-12 11:48:21.327711: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-12 11:48:21.405583: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-06-12 11:48:21.405699: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (d3931badea88): /proc/driver/nvidia/version does not exist
/content//14213352230574.jpg
	[Original] Actual size fed into NN:  (1, 360, 270, 3)
2021-06-12 11:48:31.938888: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-0

127.0.0.1 - - [12/Jun/2021 11:50:00] "POST /callback HTTP/1.1" 200 -


計算時間: 108.19 秒


127.0.0.1 - - [12/Jun/2021 11:50:44] "POST /callback HTTP/1.1" 200 -
